### 중앙치매센터 치매시설정보 크롤링
- https://www.nid.or.kr/info/facility_list.aspx

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import pymongo as mg

In [2]:
client = mg.MongoClient(host='mongodb://localhost:27017')
database = client['ns_os_reviews']
collection = database['naver_place']

In [3]:
# open chrome browser
browser = webdriver.Chrome(executable_path='../../../../chromedriver.exe')

In [4]:
url = 'https://www.nid.or.kr/info/facility_list.aspx'
browser.get(url)
browser.implicitly_wait(3)

### Tags
['시설명', '주소', '시설분류정보', '전화번호', '홈페이지']<br>
['name', 'address', 'category', 'number', 'website']<br>
['시설명', '주소', '위도', '경도', '대분류', '중분류', '소분류', '전화번호', '홈페이지']<br>
['name', 'address', 'latitude', 'longitude', 'main_category', 'sub_category', 'sub_category_2', 'number', 'website']
- 상세 버튼 : #dev_tbody > tr:nth-child({1~10, 마지막페이지 8}) > td:nth-child(4) > a
- 상세 버튼 목록 : #dev_tbody > tr > td:nth-child(4) > a
- 닫기 버튼 : div > div.popTitle > a

- 페이지 버튼 목록 : #div_paging > div > ol > li > a
- 페이지 버튼 : #div_paging > div > ol > li:nth-child({1~10}, 마지막 1) > a
- 다음 페이지 목록 : xpath : //*[@id="div_paging"]/div/a/img[@alt="다음"]
- 시설명 : div.list_tbl_01.fix > table > tbody > tr:nth-child(1) > td:nth-child(2)
- 주소 : div.list_tbl_01.fix > table > tbody > tr:nth-child(2) > td
- 시설분류정보 : div.list_info > div.dx_fa_div > ul > li
- 전화번호 : div.list_tbl_01.fix > table > tbody > tr:nth-child(1) > td:nth-child(4)
- 홈페이지 : div.list_tbl_01.fix > table > tbody > tr:nth-child(3) > td


In [13]:
# 페이지 버튼 목록 가져오기
while True : 
    page_list = browser.find_elements_by_css_selector('#div_paging > div > ol > li > a')
    detail_btns = browser.find_elements_by_css_selector('#dev_tbody > tr > td:nth-child(4) > a')
    
    for index, page in enumerate(page_list) :
        if index == 0 :
            for btn in detail_btns :
                btn.click()
                close_btn = browser.find_element_by_css_selector('.btnClose')
                close_btn.click()
                
           

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".btnClose"}
  (Session info: chrome=116.0.5845.188)


In [7]:
cursor = collection.find({})
list_cursors = list(cursor)

In [8]:
df_total = pd.DataFrame(list_cursors)

In [9]:
df_total[:3]

,_id,hospital,address,name,content,date,visits
0,6503f9cf15803af914b2b3f2,서울바른병원,서울 영등포구 여의대방로 145,oyu****,"입구부터 느껴지는 청결함과, 친절한 분위기! 웃으며 인사해주신 직원분께 감사",2023년 9월 14일 목요일,1번째 방문
1,6503f9cf15803af914b2b3f3,서울바른병원,서울 영등포구 여의대방로 145,o0p****,설명 쉽게 쉽게 잘해주시고 꼼꼼함 면에서도 상당히 만족스러워요,2023년 9월 13일 수요일,1번째 방문
2,6503f9cf15803af914b2b3f4,서울바른병원,서울 영등포구 여의대방로 145,asy****,"다른데 안가고 여기서 치료 받길 잘했어요, 상담부터 편하게해주셔서 감사해요",2023년 9월 12일 화요일,1번째 방문


In [10]:
df_drop_id = df_total.drop('_id', axis=1)
df_drop_id[:3]

,hospital,address,name,content,date,visits
0,서울바른병원,서울 영등포구 여의대방로 145,oyu****,"입구부터 느껴지는 청결함과, 친절한 분위기! 웃으며 인사해주신 직원분께 감사",2023년 9월 14일 목요일,1번째 방문
1,서울바른병원,서울 영등포구 여의대방로 145,o0p****,설명 쉽게 쉽게 잘해주시고 꼼꼼함 면에서도 상당히 만족스러워요,2023년 9월 13일 수요일,1번째 방문
2,서울바른병원,서울 영등포구 여의대방로 145,asy****,"다른데 안가고 여기서 치료 받길 잘했어요, 상담부터 편하게해주셔서 감사해요",2023년 9월 12일 화요일,1번째 방문


In [12]:
df_drop_id.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15672 entries, 0 to 15671
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   hospital  15672 non-null  object
 1   address   15672 non-null  object
 2   name      15672 non-null  object
 3   content   15672 non-null  object
 4   date      15672 non-null  object
 5   visits    15672 non-null  object
dtypes: object(6)
memory usage: 734.8+ KB


In [11]:
df_drop_dup = df_drop_id.drop_duplicates()
df_drop_dup.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14102 entries, 0 to 15671
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   hospital  14102 non-null  object
 1   address   14102 non-null  object
 2   name      14102 non-null  object
 3   content   14102 non-null  object
 4   date      14102 non-null  object
 5   visits    14102 non-null  object
dtypes: object(6)
memory usage: 771.2+ KB


In [13]:
df_drop_dup.to_csv('./JungMoonChoi.csv')

In [6]:
# {"content": {"$regex": "수술"}}
# {"hospital": {"$regex": "연세백퍼센트병원"}}
# {"name": {"$regex": "동동"}}